In [7]:
#Importamos la librerías y variables necesarias
from mysql.connector import pooling
import pandas as pd
from flask import Flask, request, jsonify, abort, make_response
from flask_cors import CORS
from dotenv import load_dotenv
import os
import json
import pickle

In [17]:
load_dotenv()
private_key = os.getenv("private_key")

# Configura el pool de conexiones
dbconfig = {
    "host": "test-db.cze2nnbbx5pc.eu-west-3.rds.amazonaws.com",
    "user": "admin",
    "password": "admin123",
    "database": "prueba"
}
connection_pool = pooling.MySQLConnectionPool(pool_name="mypool", pool_size=4, **dbconfig)
# Función auxiliar para obtener una conexión del pool
def get_connection():
    return connection_pool.get_connection()

In [18]:
#Funciones individuales que atacan a la base de datos
def tabla_current_employees():
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT * from current_employees")
        resultado = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        cursor.close()
        conn.close() 
        X = pd.DataFrame(resultado, columns=column_names)
        return(X)

def borrar_datos_predictions():
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("TRUNCATE TABLE predictions;")
        conn.commit()
        cursor.close()
        conn.close() 
def subir_nuevos_datos_predictions(df_final):
        conn = get_connection()
        cursor = conn.cursor()

        for _, row in df_final.iterrows():
                insert_query = f"INSERT INTO predictions (id_employee, total_months_in_company) VALUES (%s, %s)"
                values = (row['id_employee'], row['total_months_in_company'])
                cursor.execute(insert_query, values)

        conn.commit()
        cursor.close()
        conn.close() 

In [19]:
X=tabla_current_employees()


In [21]:
borrar_datos_predictions()

In [30]:
model = pickle.load(open('Data/Output/Models/Models/JP_12_06_VotingRegressor.pickle', 'rb'))
scaler = pickle.load(open('Data/Output/Models/Models/scaler.pickle', 'rb'))
columns_to_drop = ['id_employee','name', 'involvement', 'performance', 'environment', 'department', 'education', 'education_field',
        'gender', 'role', 'years_curr_manager','total_working_years', 'last_promotion', 'age', 'years_company']
ids=X["id_employee"].tolist()
X.drop(columns_to_drop, axis=1, inplace=True)
X = scaler.transform(X)
ls_new_value = model.predict(X)
df_final = pd.DataFrame({'id_employee': ids, 'total_months_in_company': ls_new_value})

In [31]:
subir_nuevos_datos_predictions(df_final)

In [1]:
# Subir la api que comprueba los headers

# Subir la api que predice y reentrena el modelo

#Conectar la FK de la base de datos

# Esquema base de datos -> 

#Esquema de todos los servicios -> Jose